In [1]:
# imports
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go

### Data Processing on WT_pKa

In [ ]:
WT_pka = pd.read_csv('WT_pka.csv')

In [ ]:
WT_pka.info()

In [ ]:
WT_pka.head()

In [ ]:
# get rid of null columns due to file 
WT_pka.drop(WT_pka.columns[-4:], axis = 1, inplace = True)
WT_pka.head()

We are going to drop more columns that we are now not interested in.

In [ ]:
WT_pka.drop(WT_pka.columns[-7:], axis = 1, inplace = True)
WT_pka.head()

In [ ]:
is_NaN = WT_pka.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = WT_pka[row_has_NaN]
print(rows_with_NaN)

# This row does not have an experimental value, so we drop it
WT_pka.dropna(inplace = True)
WT_pka.isna().sum()

In [ ]:
WT_pka['Res ID'] = WT_pka['Res ID'].astype(int)
WT_pka.head()

Process irregular values in Expt. pKa

In [ ]:
# Create a new column 'Greater/Smaller' to keep record of Expt. pKa
WT_pka['Greater/Smaller'] = 0

WT_pka.loc[WT_pka['Expt. pKa'].str.contains(">"), 'Greater/Smaller'] = 1
WT_pka.loc[WT_pka['Expt. pKa'].str.contains("<"), 'Greater/Smaller'] = -1

WT_pka['Expt. pKa'] = WT_pka['Expt. pKa'].str.replace('>', '')
WT_pka['Expt. pKa'] = WT_pka['Expt. pKa'].str.replace('<', '')
WT_pka['Expt. pKa'] = WT_pka['Expt. pKa'].str.replace('~', '')

In [ ]:
# There are two rows with two pKa valus, created a new row to store the second value
print(WT_pka[WT_pka['Expt. pKa'].str.contains(",")])
WT_pka['2nd pKa'] = 0.0
WT_pka[['Expt. pKa','2nd pKa']] = WT_pka['Expt. pKa'].str.split(',',expand=True)
WT_pka.loc[WT_pka['2nd pKa'] == 'None', '2nd pKa'] = '0'
WT_pka['Expt. pKa'] = WT_pka['Expt. pKa'].astype(float)

WT_pka['2nd pKa'] = WT_pka['2nd pKa'].astype(float)
WT_pka['2nd pKa'] = WT_pka['2nd pKa'].fillna(0)

WT_pka.info()

In [ ]:
WT_pka.head()

<hr style="border:1px solid gray"> </hr>

### Data processing on individual proteins (pKa.csv and output.pqr)

#### First create a dataframe for theoretical pka values for future use

In [2]:
# theoretical value of proteins
theo_val = {'ARG': 12.0, 'ASP': 4.0, 'CYS': 9.5, 'GLU': 4.4, 'HIS': 6.3, 
               'LYS': 10.4, 'TYR': 9.6}

df_theo_val = pd.DataFrame(np.array([['ARG', 12.0], ['ASP', 4.0], ['CYS', 9.5], 
                                    ['GLU', 4.4], ['HIS', 6.3], ['LYS', 10.4], ['TYR', 9.6]]), 
                          columns = ['Res Name', 'pKa'])
df_theo_val

,Res Name,pKa
0,ARG,12.0
1,ASP,4.0
2,CYS,9.5
3,GLU,4.4
4,HIS,6.3
5,LYS,10.4
6,TYR,9.6


<hr style="border:1px solid gray"> </hr>

### We use 2ovo as an example

#### Read 2ovo pka file 

In [ ]:
# rearrange pKa.csv, we use 2ovo as an example
df_2ovo = pd.read_csv('sample_data/2ovo/pKa.csv')
df_2ovo.info()

In [ ]:
# We see that all the columns are now in one column, so we need to split them.
df_2ovo[list(df_2ovo.columns)[0].split()] = df_2ovo.iloc[:,0].str.split(expand=True)
df_2ovo.drop(df_2ovo.columns[0], axis = 1, inplace = True)

# Split the Res ID and Res Name from ResName
# "(?<=\\D)(?=\\d)|(?<=\\d)(?=\\D)" split digits and chars
df_2ovo[['Res Name', 'Res ID', 'Chain']] = df_2ovo.iloc[:,0].str.split("(?<=\\D)(?=\\d)|(?<=\\d)(?=\\D)", expand=True)
df_2ovo.drop(df_2ovo.columns[0], axis = 1, inplace = True)
df_2ovo['Res ID'] = df_2ovo['Res ID'].astype(int)
df_2ovo = df_2ovo[list(df_2ovo.columns)[-3:-1]+ list(df_2ovo.columns)[0:-3]]
df_2ovo = df_2ovo[list(df_2ovo.columns)[0:3]]

df_2ovo.head()

In [ ]:
# Merge with theoretical values
df_2ovo.rename(columns={"pKa": "Expt. pKa"}, inplace=True)
df_2ovo = pd.merge(df_2ovo, df_theo_val, on=['Res Name'], how='inner')
df_2ovo

#### Read 2ovo pqr file 

In [ ]:
file = open('sample_data/2ovo/output.pqr', 'r')
lines = file.readlines()
lines = lines[:-1]
file.close()
column_names = ['Res ID', 'x', 'y', 'z', 'Charge', 'Radius']
df_2ovo_pqr = pd.DataFrame(columns=column_names)
target_IDs = list(df_2ovo['Res ID'].unique().astype(int))
print(target_IDs)
i = 0
for line in lines:
    line = line.strip().split()
    if int(line[5]) in target_IDs:
        df_2ovo_pqr.loc[i] = line[5:] 
        i += 1
df_2ovo_pqr['Res ID'] = df_2ovo_pqr['Res ID'].astype(int)
df_2ovo_pqr[['x', 'y', 'z', 'Charge', 'Radius']] = df_2ovo_pqr[['x', 'y', 'z', 'Charge', 'Radius']].astype(float)
df_2ovo_pqr.head()

In [ ]:
df_2ovo = pd.merge(df_2ovo, df_2ovo_pqr, on=['Res ID'], how='inner')
df_2ovo.head()

In [ ]:

fig = go.Figure()

res_IDs = list(df_2ovo['Res ID'].unique())
data = []

for ID in res_IDs:
    res_name = list(df_2ovo.loc[(df_2ovo['Res ID']) == ID,'Res Name'].unique())[0]
    trace = go.Scatter3d(
        x=df_2ovo.loc[(df_2ovo['Res ID']) == ID,'x'],
        y=df_2ovo.loc[(df_2ovo['Res ID']) == ID,'y'],
        z=df_2ovo.loc[(df_2ovo['Res ID']) == ID,'z'],

        mode='markers',
        marker=dict(
            size=3,
            colorscale='Viridis',   
        ),
        name= res_name+' '+str(ID),

        # list comprehension to add text on hover
        text= [f"x: {a}<br>y: {b}<br>z: {c}" for a,b,c in list(zip(df_2ovo['x'], df_2ovo['y'], df_2ovo['z']))],
        # if you do not want to display x,y,z
        hoverinfo='text'
    )
    fig.add_trace(trace)
    data.append(trace)

layout = dict(title = 'TEST',)

F = dict(data=data, layout=layout)
py.offline.plot(F, filename = 'Test.html')


<hr style="border:1px solid gray"> </hr>

### For any PDBID

In [26]:
def read_csv(PDBID):
    df_PDB_csv = pd.read_csv('sample_data/' + PDBID.lower() + '/pKa.csv')
    
    # We see that all the columns are now in one column, so we need to split them.
    df_PDB_csv[list(df_PDB_csv.columns)[0].split()] = df_PDB_csv.iloc[:,0].str.split(expand=True)
    df_PDB_csv.drop(df_PDB_csv.columns[0], axis = 1, inplace = True)

    # Split the Res ID and Res Name from ResName
    # "(?<=\\D)(?=\\d)|(?<=\\d)(?=\\D)" split digits and chars
    df_PDB_csv[['Res Name', 'Res ID', 'Chain']] = df_PDB_csv.iloc[:,0].str.split("(?<=\\D)(?=\\d)|(?<=\\d)(?=\\D)", expand=True)
    df_PDB_csv.drop(df_PDB_csv.columns[0], axis = 1, inplace = True)
    df_PDB_csv['Res ID'] = df_PDB_csv['Res ID'].astype(int)
    df_PDB_csv = df_PDB_csv[list(df_PDB_csv.columns)[-3:-1]+ list(df_PDB_csv.columns)[0:-3]]
    df_PDB_csv = df_PDB_csv[list(df_PDB_csv.columns)[0:3]]
    
    # merge with theoretical values
    df_PDB_csv.rename(columns={"pKa": "Expt. pKa"}, inplace=True)
    df_PDB_csv = pd.merge(df_PDB_csv, df_theo_val, on=['Res Name'], how='inner')
    
    df_PDB_csv['Expt. pKa'] = df_PDB_csv['Expt. pKa'].astype(float)
    df_PDB_csv['pKa'] = df_PDB_csv['pKa'].astype(float)
    
    
    return df_PDB_csv

In [19]:
def read_pqr(PDBID, df_PDB_csv = None, flag = False):
    file = open('sample_data/' + PDBID.lower() + '/output.pqr', 'r')
    lines = file.readlines()
    lines = lines[:-1]
    file.close()
    
    column_names = ['Atom Name', 'Res Name', 'Res ID', 'x', 'y', 'z', 'Charge', 'Radius']
    df_PDB_pqr = pd.DataFrame(columns=column_names)
    if flag:
        target_IDs = list(df_PDB_csv['Res ID'].unique().astype(int))

    i = 0
    
    # find corresponding res ID in pqr file
    for line in lines:
        line = line.strip().split()
        if len(line) == 11:
            if flag == False:
                df_PDB_pqr.loc[i] = [line[2]] + [line[3]] + line[5:]
            elif ((flag) & (int(line[5]) in target_IDs)):
                df_PDB_pqr.loc[i] = [line[2]] + [line[3]] + line[5:]
            i += 1
            
    # convert datatype
    df_PDB_pqr['Res ID'] = df_PDB_pqr['Res ID'].astype(int)
    df_PDB_pqr[['x', 'y', 'z', 'Charge', 'Radius']] = df_PDB_pqr[['x', 'y', 'z', 'Charge', 'Radius']].astype(float)
    df_PDB_pqr.head()
    return df_PDB_pqr
    

## Visualization on a protein

In [130]:
def plot_PDB(PDBID, df_PDB):
    
    fig = go.Figure()

    res_IDs = list(df_PDB['Res ID'].unique())
    data = []

    for ID in res_IDs:
        res_name = list(df_PDB.loc[(df_PDB['Res ID']) == ID,'Res Name'].unique())[0]
        trace = go.Scatter3d(
            x=df_PDB.loc[(df_PDB['Res ID']) == ID,'x'],
            y=df_PDB.loc[(df_PDB['Res ID']) == ID,'y'],
            z=df_PDB.loc[(df_PDB['Res ID']) == ID,'z'],

            mode='markers',
            marker=dict(
                size=3,
                colorscale='Viridis',   
            ),
            name = res_name + ' ' + str(ID),
            # list comprehension to add text on hover
            text = [f"x: {a}<br>y: {b}<br>z: {c}<br>res: {d}" 
                   for a,b,c,d in list(zip(df_PDB['x'], df_PDB['y'], df_PDB['z'], [res_name + ' ' + str(ID)]*len(df_PDB['x']))) ],
            # if you do not want to display x,y,z
            hoverinfo='text'
        )
        fig.add_trace(trace)
        data.append(trace)

    layout = dict(title = PDBID.upper(),)

    F = dict(data=data, layout=layout)
    py.offline.plot(F, filename = 'sample_graphs/' +PDBID + '.html')
    

In [144]:
def analyze_PDB(PDBID):
    df_PDB_csv = read_csv(PDBID)
    df_PDB_pqr = read_pqr(PDBID, df_PDB_csv, flag = True)
    # merge csv and pqr
    df_PDB = pd.merge(df_PDB_csv, df_PDB_pqr, on=['Res ID', 'Res Name'], how='inner')
    plot_PDB(PDBID, df_PDB)

In [157]:
sample_data = ['1bf4', '1bpi', '1igd', '1pga', '1pgb', '2ci2', '2ovo', '2qmt', '3ebx', '4pti']
for PDBID in sample_data:
    analyze_PDB(PDBID)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 507 entries, 0 to 506
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Res Name   507 non-null    object 
 1   Res ID     507 non-null    int64  
 2   Expt. pKa  507 non-null    object 
 3   pKa        507 non-null    object 
 4   Atom Name  507 non-null    object 
 5   x          507 non-null    float64
 6   y          507 non-null    float64
 7   z          507 non-null    float64
 8   Charge     507 non-null    float64
 9   Radius     507 non-null    float64
dtypes: float64(5), int64(1), object(4)
memory usage: 43.6+ KB


<hr style="border:1px solid gray"> </hr>

### Preprocess Data for Prediction
- Purpose here is to analyze the same amino acid and see if there's a pattern even amoung different proteins
- We will first use LYS and our sample_data as an experiment. Things we need to do:
    - Calculate Coulomb force on each LYS atom from all the other atoms (since looping in python is terrible, we might use matrix?)
    - We need to extract all rows of LYS from our sample proteins
    - The features that we are interested in are 'Atom name', 'Res Name', 'Res ID', 'x', 'y', 'z', 'Charge', 'Radius'
    - One observation is that for the same atom, its charge and radius are the same. 
        - Need to confirm if it's true
        - We could analyze whether the prediction behaves differently if we replace the numerical value with only the atom if we decide whether it is discrete or continuous

In [20]:
target_AA = 'ASP'
sample_data = ['1bf4', '1bpi', '1igd', '1pga', '1pgb', '2ci2', '2ovo', '2qmt', '3ebx', '4pti']

In [21]:
def extract_with_AA(PDBID, Amino_Acid):
    df_PDB_pqr_all = read_pqr(PDBID)
    df_PDB_csv = read_csv(PDBID)
    df_PDB_pqr = read_pqr(PDBID, df_PDB_csv)
    df_PDB = pd.merge(df_PDB_csv, df_PDB_pqr, on=['Res ID', 'Res Name'], how='inner')
    target_rows = df_PDB.loc[(df_PDB['Res Name'] == Amino_Acid)]
    return target_rows, df_PDB_pqr_all

In [22]:
def calculate_coulomb_force(target_row, df_PDB_pqr_all):
    coulomb_force = 0
    x = target_row['x']
    y = target_row['y']
    z = target_row['z']
    v_target = np.array((x,y,z))
    for index, row in df_PDB_pqr_all.iterrows():
        v = np.array((row['x'], row['y'], row['z']))
        d = v_target - v
        dist = (d @ d)**.5
#         dist = np.sqrt((row['x'] - x)**2 + (row['y'] - y)**2 + (row['z'] - z)**2)
        if dist == 0:
            continue
        coulomb_force = coulomb_force + row['Charge']/dist
    return coulomb_force

In [23]:
def arrange_df(PDBID):
    df_PDB, df_PDB_pqr_all = extract_with_AA(PDBID, target_AA)
    df_PDB['Columb Force'] = 0
    for index, row in df_PDB.iterrows():
        df_PDB.loc[index, 'Columb Force'] = calculate_coulomb_force(row, df_PDB_pqr_all)
    df_PDB['PDBID'] = PDBID.upper()
    return df_PDB

In [24]:
# find the same Amino Acid in all PDB
def concat_DFs(sample_data):
    first = True
    for PDBID in sample_data:
        df_PDB = arrange_df(PDBID)
        # rearrange columns
        df_PDB = df_PDB[[list(df_PDB.columns)[-1]] + [list(df_PDB.columns)[4]] + 
                          list(df_PDB.columns)[0:2] + list(df_PDB.columns)[-7:-1] + 
                          list(df_PDB.columns)[2:4]]
        if first:
            df_AA = pd.concat([df_PDB])
            first = False
        else:
            df_AA = pd.concat([df_AA, df_PDB])
    return df_AA

In [27]:
%%time
df_ASP = concat_DFs(sample_data)

CPU times: user 1min 20s, sys: 79.3 ms, total: 1min 20s
Wall time: 1min 20s


In [28]:
pd.set_option('display.max_rows', None)
df_ASP.reset_index(drop=True)
df_ASP = df_ASP.reset_index(drop=True)

In [29]:
df_ASP

,PDBID,Atom Name,Res Name,Res ID,x,y,z,Charge,Radius,Columb Force,Expt. pKa,pKa
0,1BF4,N,ASP,16,10.613,14.713,5.161,-0.5163,1.824,0.208499,3.84,4.0
1,1BF4,H,ASP,16,11.476,13.552,5.558,0.2936,0.600,-0.472558,3.84,4.0
2,1BF4,CA,ASP,16,9.957,14.708,3.853,0.0381,1.908,-0.332752,3.84,4.0
3,1BF4,HA,ASP,16,10.290,15.551,3.280,0.0880,1.387,-0.399930,3.84,4.0
4,1BF4,C,ASP,16,8.445,14.776,4.078,0.5366,1.908,-0.923343,3.84,4.0
5,1BF4,O,ASP,16,7.899,14.039,4.900,-0.5819,1.661,0.194376,3.84,4.0
6,1BF4,CB,ASP,16,10.312,13.439,3.071,-0.0303,1.908,-0.220848,3.84,4.0
7,1BF4,HB1,ASP,16,9.713,12.653,3.487,-0.0122,1.487,-0.253987,3.84,4.0
8,1BF4,HB2,ASP,16,11.381,13.365,3.090,-0.0122,1.487,-0.299017,3.84,4.0
9,1BF4,CG,ASP,16,9.923,13.541,1.622,0.7994,1.908,-1.394836,3.84,4.0


In [30]:
df_ASP_copy = df_ASP.copy()

In [31]:
df_ASP_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PDBID         442 non-null    object 
 1   Atom Name     442 non-null    object 
 2   Res Name      442 non-null    object 
 3   Res ID        442 non-null    int64  
 4   x             442 non-null    float64
 5   y             442 non-null    float64
 6   z             442 non-null    float64
 7   Charge        442 non-null    float64
 8   Radius        442 non-null    float64
 9   Columb Force  442 non-null    float64
 10  Expt. pKa     442 non-null    float64
 11  pKa           442 non-null    float64
dtypes: float64(8), int64(1), object(3)
memory usage: 41.6+ KB


In [32]:
df_ASP_copy['Diff'] = 0
df_ASP_copy['Diff'] = df_ASP_copy['Expt. pKa'] - df_ASP_copy['pKa']

In [34]:
df_ASP_copy['Diff'].describe()

count    442.000000
mean      -0.518529
std        0.381631
min       -1.230000
25%       -0.880000
50%       -0.515000
75%       -0.130000
max        0.030000
Name: Diff, dtype: float64

In [55]:
pivot = (max(df_ASP_copy['Expt. pKa']) + min(df_ASP_copy['Expt. pKa']))/2
print(pivot)

3.4000000000000004


In [56]:
df_ASP_copy['Target'] = 0
df_ASP_copy.loc[df_ASP_copy['Expt. pKa'] >= pivot, 'Target'] = 1
df_ASP_copy.loc[df_ASP_copy['Expt. pKa'] < pivot, 'Target'] = 0

In [57]:
df_ASP_copy.head()

,PDBID,Atom Name,Res Name,Res ID,x,y,z,Charge,Radius,Columb Force,Expt. pKa,pKa,Diff,Target
0,1BF4,N,ASP,16,10.613,14.713,5.161,-0.5163,1.824,0.208499,3.84,4.0,-0.16,1
1,1BF4,H,ASP,16,11.476,13.552,5.558,0.2936,0.600,-0.472558,3.84,4.0,-0.16,1
2,1BF4,CA,ASP,16,9.957,14.708,3.853,0.0381,1.908,-0.332752,3.84,4.0,-0.16,1
3,1BF4,HA,ASP,16,10.290,15.551,3.280,0.0880,1.387,-0.399930,3.84,4.0,-0.16,1
4,1BF4,C,ASP,16,8.445,14.776,4.078,0.5366,1.908,-0.923343,3.84,4.0,-0.16,1


<hr style="border:1px solid gray"> </hr>

#### Logistic regression: predict of Target

In [70]:
from sklearn.model_selection import train_test_split
df_ASP_copy.rename(columns={"Columb Force": "Columb_Force"}, inplace=True)

train, test = train_test_split(df_ASP_copy, test_size=0.2)

In [98]:
feature_columns = ['x', 'y', 'z', 'Charge', 'Radius', 'Columb_Force']
train_x = train[feature_columns]
train_y = train['Target']

test_x = test[feature_columns]
test_y = test['Target']

In [108]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_sk = LogisticRegression(solver='liblinear') # all params default

lr_sk.fit(train_x,train_y)
print(np.hstack((lr_sk.intercept_[:,np.newaxis],lr_sk.coef_)))
yhat = lr_sk.predict(test_x)
print('Accuracy of: ', accuracy_score(test_y, yhat))

[[-0.39789013  0.02943269 -0.00616538 -0.06898261 -3.05859916  0.42181701
  -3.62835547]]
Accuracy of:  0.8539325842696629


#### DNN

In [80]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

In [81]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [82]:
my_feature_columns = []
for key in X.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='y', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='z', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Charge', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Radius', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Columb_Force', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [92]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/bg/sjf18vvj1l30vdgzyw_lhnhw0000gn/T/tmpssf2fkpz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [93]:
classifier.train(
    input_fn=lambda: input_fn(X, y, training=True),
    steps=5000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/bg/sjf18vvj1l30vdgzyw_lhnhw0000gn/T/tmpssf2fkpz/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INF

In [94]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test[feature_columns], test['Target'], training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-03-05T17:07:56Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/bg/sjf18vvj1l30vdgzyw_lhnhw0000gn/T/tmpssf2fkpz/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.49851s
INFO:tensorflow:Finished evaluation at 2021-03-05-17:07:56
INFO:tensorflow:Saving dict for glo

<hr style="border:1px solid gray"> </hr>

<hr style="border:1px solid gray"> </hr>

<hr style="border:1px solid gray"> </hr>

<hr style="border:1px solid gray"> </hr>